In [2]:
import tensorflow as tf
from pathlib import Path

In [9]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets/", extract=True)

extracted_path = Path(path).with_name("spa-eng") / "spa.txt"
text = extracted_path.read_text(encoding="utf-8")

print(text[:500])

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.
Run!	¡Corre!
Run.	Corred.
Who?	¿Quién?
Fire!	¡Fuego!
Fire!	¡Incendio!
Fire!	¡Disparad!
Help!	¡Ayuda!
Help!	¡Socorro! ¡Auxilio!
Help!	¡Auxilio!
Jump!	¡Salta!
Jump.	Salte.
Stop!	¡Parad!
Stop!	¡Para!
Stop!	¡Pare!
Wait!	¡Espera!
Wait.	Esperen.
Go on.	Continúa.
Go on.	Continúe.
Hello!	Hola.
I ran.	Corrí.
I ran.	Corría.
I try.	Lo intento.
I won!	¡He ganado!
Oh no!	¡Oh, no!
Relax.	Tomátelo con soda.
Smile.	Sonríe.
Attack!	¡Al ataque!
Attack!	¡Atacad!
Ge


In [13]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42) 
np.random.shuffle(pairs)

sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [14]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [15]:
vocab_size = 1000
max_length = 50

text_vec_layer_en = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)

text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [16]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [17]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [25]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])

X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant(["startofseq {s}" for s in sentences_es[100_000:]])

y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [19]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

#### Building Model Using Functional API

In [22]:
embed_size = 128

encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)

encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [21]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [23]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [24]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
y_proba = output_layer(decoder_outputs)

In [26]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit((X_train, X_train_dec), y_train, epochs=5, validation_data=((X_valid, X_valid_dec), y_valid))

Epoch 1/5
3125/3125 [==============================] - 3327s 1s/step - loss: 2.9674 - accuracy: 0.4189 - val_loss: 3.0684 - val_accuracy: 0.4193
Epoch 2/5
3125/3125 [==============================] - 3065s 981ms/step - loss: 1.8857 - accuracy: 0.5677 - val_loss: 2.5862 - val_accuracy: 0.4599
Epoch 3/5
3125/3125 [==============================] - 3068s 982ms/step - loss: 1.4596 - accuracy: 0.6444 - val_loss: 2.4194 - val_accuracy: 0.4765
Epoch 4/5
3125/3125 [==============================] - 3067s 981ms/step - loss: 1.2184 - accuracy: 0.6911 - val_loss: 2.3970 - val_accuracy: 0.4853
Epoch 5/5
3125/3125 [==============================] - 3078s 985ms/step - loss: 1.0548 - accuracy: 0.7247 - val_loss: 2.4538 - val_accuracy: 0.4805


#### Translating English to Spanish Language

In [35]:
def translate(sentence_en):
    translation = ""
    for word_inx in range(max_length):
        X = np.array([sentence_en])
        X_dec = np.array(["startofseq" + translation])
        
        y_proba = model.predict((X, X_dec), verbose=0)[0, word_inx]
        
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
        
    return translation

In [36]:
translate("I love you")

' te amo'

In [37]:
translate("I love soccer")

' me encanta el fútbol'

In [38]:
translate("I am going to school right now.")

' voy a ir al colegio ahora mismo'

#### Bidirectional RNNs

In [41]:
tf.random.set_seed(42)

encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True)
)

In [42]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

In [44]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
y_proba = output_layer(decoder_outputs)

model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit((X_train, X_train_dec), y_train, epochs=5, validation_data=((X_valid, X_valid_dec), y_valid))

Epoch 1/5
3125/3125 [==============================] - 2378s 756ms/step - loss: 2.6217 - accuracy: 0.4659 - val_loss: 2.8547 - val_accuracy: 0.4315
Epoch 2/5
3125/3125 [==============================] - 2367s 758ms/step - loss: 1.6265 - accuracy: 0.6139 - val_loss: 2.5133 - val_accuracy: 0.4666
Epoch 3/5
3125/3125 [==============================] - 2369s 758ms/step - loss: 1.2939 - accuracy: 0.6767 - val_loss: 2.3738 - val_accuracy: 0.4832
Epoch 4/5
3125/3125 [==============================] - 2371s 759ms/step - loss: 1.1069 - accuracy: 0.7145 - val_loss: 2.4069 - val_accuracy: 0.4800
Epoch 5/5
3125/3125 [==============================] - 2364s 756ms/step - loss: 0.9702 - accuracy: 0.7427 - val_loss: 2.3451 - val_accuracy: 0.4887


#### Beam Search: A very basic implementation of beam search

In [45]:
def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # encoder input
    X_dec = np.array(["startofseq"])  # decoder input
    y_proba = model.predict((X, X_dec))[0, 0]  # first token's probas
    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # list of best (log_proba, translation)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]
    
    # displays the top first words in verbose mode
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # translation is finished, so don't try to extend it
            X = np.array([sentence_en])  # encoder input
            X_dec = np.array(["startofseq " + translation])  # decoder input
            y_proba = model.predict((X, X_dec))[0, idx]  # last token's proba
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append((log_proba + np.log(word_proba),
                                   f"{translation} {word}"))
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # displays the top translation so far in verbose mode
        if verbose:
            print("Top translations so far:", top_translations)

        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()

In [46]:
sentence_en = "I love cats and dogs"
translate(sentence_en)

' amo a los perros y gatos'

In [47]:
beam_search(sentence_en, beam_width=3, verbose=True)

1/1 [==============================] - 0s 62ms/step
Top first words: [(-0.6904845, 'amo'), (-1.4752278, 'me'), (-2.326438, 'conozco')]
1/1 [==============================] - 0s 50ms/step
Top translations so far: [(-0.91566813, 'amo a'), (-1.8409061, 'me [UNK]'), (-2.6258357, 'conozco a')]
1/1 [==============================] - 0s 50ms/step
Top translations so far: [(-1.0702174, 'amo a los'), (-2.1794553, 'me [UNK] los'), (-2.8969083, 'conozco a los')]
1/1 [==============================] - 0s 50ms/step
Top translations so far: [(-1.7575715, 'amo a los perros'), (-1.8192847, 'amo a los gatos'), (-2.6674814, 'me [UNK] los gatos')]
1/1 [==============================] - 0s 52ms/step
Top translations so far: [(-1.9208897, 'amo a los gatos y'), (-2.0812309, 'amo a los perros y'), (-2.7925155, 'me [UNK] los gatos y')]
1/1 [==============================] - 0s 50ms/step
Top translations so far: [(-2.353173, 'amo a los perros y gatos'), (-2.3917155, 'amo a los gatos y perros'), (-2.8942475, 'm

'amo a los perros y gatos'